In [1]:
!nvidia-smi

Tue Jul  9 16:24:25 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk, load_metric
import matplotlib.pyplot as plt
import pandas as pd

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_ckpt = "google/pegasus-cnn_dailymail"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
# Download the dataset
!wget https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip
!unzip summarizer-data.zip

--2024-07-09 16:20:03--  https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/entbappy/Branching-tutorial/master/summarizer-data.zip [following]
--2024-07-09 16:20:04--  https://raw.githubusercontent.com/entbappy/Branching-tutorial/master/summarizer-data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7903594 (7.5M) [application/zip]
Saving to: ‘summarizer-data.zip.1’

summarizer-data.zip 100%[===================>]   7.54M  --.-KB/s    in 0.02s   

2024-07-09 16:20:04 (309 MB/s) - ‘summarizer-data.zip.1’ saved [7903

In [ ]:
!ls -all

total 17948
drwxr-xr-x 1 root root    4096 Jul  9 15:43 .
drwxr-xr-x 1 root root    4096 Jul  9 15:23 ..
drwxr-xr-x 4 root root    4096 Jul  3 13:21 .config
drwxr-xr-x 1 root root    4096 Jul  3 13:21 sample_data
drwxr-xr-x 5 root root    4096 May  9  2023 samsum_dataset
-rw-r--r-- 1 root root  530165 May  9  2023 samsum-test.csv
-rw-r--r-- 1 root root 9401418 May  9  2023 samsum-train.csv
-rw-r--r-- 1 root root  512131 May  9  2023 samsum-validation.csv
-rw-r--r-- 1 root root 7903594 Jul  9 15:43 summarizer-data.zip


In [5]:
data_samsum = load_from_disk('samsum_dataset')

In [6]:
data_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [7]:
split_lengths = [len(data_samsum[split]) for split in data_samsum]

print(f"Split lengths: {split_lengths}")
print(f"Features: {data_samsum['train'].column_names}")
print("\nDialogue:")

print(data_samsum['test'][1]['dialogue'])

print("\nSummary:")

print(data_samsum['test'][1]['summary'])

Split lengths: [14732, 819, 818]
Features: ['id', 'dialogue', 'summary']

Dialogue:
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)

Summary:
Eric and Rob are going to watch a stand-up on youtube.


In [8]:
def convert_examples_to_features(example_batch):
    """
    Tokenize the raw text
    """
    input_encodings = tokenizer(example_batch['dialogue'], max_length=1024, truncation=True)

    target_encodings = tokenizer(text_target=example_batch['summary'], max_length=128, truncation=True)

    return {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

In [9]:
data_samsum_pt = data_samsum.map(convert_examples_to_features, batched=True)

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

In [10]:
data_samsum_pt['train']

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [13]:
# Training

from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

In [14]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir='pegasus-samsum',
    num_train_epochs=1,
    warmup_steps=500,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    logging_steps=10,
    eval_strategy='steps',
    eval_steps=500,
    save_steps=1e6,
    gradient_accumulation_steps=16
)

In [15]:
trainer = Trainer(
    model=model_pegasus,
    args=trainer_args,
    tokenizer=tokenizer,
    data_collator=seq2seq_data_collator,
    train_dataset=data_samsum_pt['test'],
    eval_dataset=data_samsum_pt['validation']
)

In [16]:
torch.set_float32_matmul_precision('high')

In [17]:
trainer.train()

Step,Training Loss,Validation Loss


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


TrainOutput(global_step=51, training_loss=3.0043694084765864, metrics={'train_runtime': 183.534, 'train_samples_per_second': 4.462, 'train_steps_per_second': 0.278, 'total_flos': 313450454089728.0, 'train_loss': 3.0043694084765864, 'epoch': 0.9963369963369964})

In [30]:
# Evaluation

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """
    split the dataset into smaller batches that we can process simultaneously.
    Yield successive batch-sized chunks from list_of_elements.
    """
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i+batch_size]


def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                                batch_size=16, device=device,
                                column_text="article",
                                column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)
    ):

        inputs = tokenizer(article_batch, max_length=1024, truncation=True,
                        padding='max_length', return_tensors="pt")

        # parameter for length penalty ensures that the model does not generate seqeunces too long
        summaries = model.generate(input_ids=inputs['input_ids'].to(device),
                                attention_mask=inputs['attention_mask'].to(device),
                                length_penalty=0.8, num_beams=8, max_length=128)

        # Finally, we decode the generated texts,
        # replace the token, and add the decoded texts with the references to the metric
        decoded_summaries = [
            tokenizer.decode(s, skip_special_tokens=True,
                            clean_up_tokenization_spaces=True)
            for s in summaries
        ]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]

        metric.add_batch(predictions=decoded_summaries, references=target_batch)
        #print("\nDecoded Summaries:")
        #print(decoded_summaries)
        #print("\nTarget Batches")
        #print(target_batches)
    # Finally compute and return the ROUGE scores
    score = metric.compute()
    return score



In [22]:
rouge_names = ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']
rouge_metric = load_metric('rouge')

In [32]:
score = calculate_metric_on_test_ds(
    data_samsum['test'][0:10],
    rouge_metric,
    trainer.model,
    tokenizer,
    batch_size=2,
    column_text='dialogue',
    column_summary='summary'
)



100%|██████████| 5/5 [00:12<00:00,  2.58s/it]


In [35]:
rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)

pd.DataFrame(rouge_dict, index=[f'pegasus'])

,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.020221,0.0,0.020213,0.020284


In [36]:
# Save model
model_pegasus.save_pretrained("pegasus-samsum-model")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


In [37]:
# Save tokenizer
tokenizer.save_pretrained('tokenizer')

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [38]:
# Load

tokenizer = AutoTokenizer.from_pretrained("/content/tokenizer")

In [43]:
# Prediction

gen_kwargs = {
    "length_penalty": 0.8,
    "num_beams": 8,
    "max_length": 128
}

pipe = pipeline('summarization',
                model='pegasus-samsum-model',
                tokenizer=tokenizer,
                device=device)

In [42]:
sample_text = data_samsum['test'][2]['dialogue']

reference = data_samsum['test'][2]['summary']

print("Dialogue:")
print(sample_text)

print("\nReference Summary:")
print(reference)

print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])



Dialogue:
Lenny: Babe, can you help me with something?
Bob: Sure, what's up?
Lenny: Which one should I pick?
Bob: Send me photos
Lenny:  <file_photo>
Lenny:  <file_photo>
Lenny:  <file_photo>
Bob: I like the first ones best
Lenny: But I already have purple trousers. Does it make sense to have two pairs?
Bob: I have four black pairs :D :D
Lenny: yeah, but shouldn't I pick a different color?
Bob: what matters is what you'll give you the most outfit options
Lenny: So I guess I'll buy the first or the third pair then
Bob: Pick the best quality then
Lenny: ur right, thx
Bob: no prob :)

Reference Summary:
Lenny can't decide which trousers to buy. Bob advised Lenny on that topic. Lenny goes with Bob's advice to pick the trousers that are of best quality.

Model Summary:
Lenny asks Bob which pair of trousers he should pick .<n>Bob says he already has purple trousers .<n>Lenny says he'll buy the first or the third pair of trousers .
